In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import pytorch_utils
DATA_DIR = 'data/'

[PyTorch-Utils]: Loading anchor candidate data...
[PyTorch-Utils]: Loading item_id to statement embedding data...


KeyboardInterrupt: 

In [ ]:
# is cuda available?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
dataset = pytorch_utils.EntityDataset(device='cuda')
test_dataset = pytorch_utils.EntityDataset(train=False, device='cuda')

[PyTorch-Utils]: Loading train set...


c:\Users\aybar\Documents\CS423-Project-3\pytorch_utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_data_no_doc_start['entity_loc'] = self.train_data_no_doc_start.groupby(['doc_id']).cumcount()


[PyTorch-Utils]: Now generating context embeddings...


100%|██████████| 18288/18288 [02:08<00:00, 142.03it/s]


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/287 [00:00<?, ?it/s]

Batches:   0%|          | 0/287 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/287 [00:00<?, ?it/s]

Entity Length: 9166
Entity shape: torch.Size([9166, 768])
[PyTorch-Utils]: Now calculating syntactic neighbours...


In [ ]:
syntax_candidates = test_dataset.syntax_candidates[-1]
syntax_candidates = [candidate_id['corpus_id'] for candidate_id in syntax_candidates]
pytorch_utils.wiki_items.iloc[syntax_candidates]

,source_item_id,information,source_item_title
596464,4934728,country of citizenship United Kingdom sport As...,Bobby Ayre
596653,4935310,country of citizenship United Kingdom sport As...,Bobby Lumley
69069,171583,country of citizenship United Kingdom sport As...,Bobby Charlton
841390,10552652,country of citizenship United Kingdom country ...,Bobby Herbert
925819,18719433,country of citizenship United Kingdom sport As...,Bobby Goldthorpe
902823,16730249,sport Association football instance of Human m...,Bobby Jeffrey
596557,4934995,country of citizenship United Kingdom country ...,Bobby Finan
246384,832731,country of citizenship United Kingdom sport As...,Bobby Campbell (English footballer)
596477,4934758,country of citizenship United Kingdom country ...,Bobby Beattie
839271,10462531,country of citizenship United Kingdom sport As...,Bobby Gough


In [ ]:
pytorch_utils.delete_corpus_embeds()

[PyTorch-Utils]: Deleting corpus embeddings...


In [ ]:
import numpy as np
def collate_fn_train(batch, entity_embeddings, context_embeddings, syntax_candidates_list, device='cuda'):
    batch_context_embeddings = []
    batch_entity_embeddings = []
    candidate_ids_batch = []
    candidate_description_embeddings_batch = []
    labels_batch = []
    for i, data in enumerate(batch):
        index, full_mention, full_mention_longest, item_id = data
        if full_mention in pytorch_utils.anchor_to_candidate:
            candidate_ids = pytorch_utils.anchor_to_candidate[full_mention].copy()
        else:
            candidate_ids = []

        if full_mention_longest in pytorch_utils.anchor_to_candidate:
            candidate_ids += pytorch_utils.anchor_to_candidate[full_mention_longest].copy()

        
        syntax_candidates = syntax_candidates_list[index]

        candidate_ids += [pytorch_utils.wiki_items.iloc[candidate_id['corpus_id']]['source_item_id'] for candidate_id in syntax_candidates]

        # Remove duplicates
        candidate_ids = list(set(candidate_ids))

        # Shuffle the candidate_ids
        np.random.shuffle(candidate_ids)

        # do we have the ground truth in candidate_ids?
        try:
            label = candidate_ids.index(item_id)
        except ValueError:
            # skip this sample
            continue
        
        candidate_description_embeddings = [
            pytorch_utils.statement_embeddings[pytorch_utils.statement_item_id_to_row[candidate_id]] if candidate_id in pytorch_utils.statement_item_id_to_row
            else pytorch_utils.description_embeddings[pytorch_utils.description_item_id_to_row[candidate_id]]
            for candidate_id in candidate_ids
        ]

        # pad candidate_ids with 0s
        candidate_ids += [0] * (15 - len(candidate_ids))
        candidate_description_embeddings += torch.zeros(15 - len(candidate_description_embeddings), 384)

        candidate_description_embeddings = torch.stack(candidate_description_embeddings)
        
        batch_context_embeddings.append(context_embeddings[index])
        batch_entity_embeddings.append(entity_embeddings[index])
        candidate_ids_batch.append(candidate_ids)
        candidate_description_embeddings_batch.append(candidate_description_embeddings)

        labels_batch.append(label)

    # labels is a list of integers, convert it to a tensor
    labels_batch = torch.tensor(labels_batch, dtype=torch.long, device=device)
    # batch_context_embeddings is a list of tensors thus we have to use stack
    batch_context_embeddings = torch.stack(batch_context_embeddings)
    batch_entity_embeddings = torch.stack(batch_entity_embeddings)
    # candidate_ids_batch is a list of lists, convert it to a tensor
    candidate_ids_batch = torch.tensor(candidate_ids_batch, dtype=torch.long, device=device)
    candidate_description_embeddings_batch = torch.stack(candidate_description_embeddings_batch)

    # move everything to device
    batch_context_embeddings = batch_context_embeddings.to(device)
    batch_entity_embeddings = batch_entity_embeddings.to(device)
    candidate_ids_batch = candidate_ids_batch.to(device)
    candidate_description_embeddings_batch = candidate_description_embeddings_batch.to(device)
    labels_batch = labels_batch.to(device)

    return batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch, labels_batch

def collate_fn_test(batch, entity_embeddings, context_embeddings, syntax_candidates_list, device='cuda'):
    batch_context_embeddings = []
    batch_entity_embeddings = []
    candidate_ids_batch = []
    candidate_description_embeddings_batch = []
    for i, data in enumerate(batch):
        index, full_mention, full_mention_longest = data
        if full_mention in pytorch_utils.anchor_to_candidate:
            candidate_ids = pytorch_utils.anchor_to_candidate[full_mention].copy()
        else:
            candidate_ids = []

        if full_mention_longest in pytorch_utils.anchor_to_candidate:
            candidate_ids += pytorch_utils.anchor_to_candidate[full_mention_longest].copy()    
        
        syntax_candidates = syntax_candidates_list[index]

        candidate_ids += [pytorch_utils.wiki_items.iloc[candidate_id['corpus_id']]['source_item_id'] for candidate_id in syntax_candidates]

        # Remove duplicates and limit to 8 candidates
        candidate_ids = list(set(candidate_ids))

        # Shuffle the candidate_ids
        np.random.shuffle(candidate_ids)

        candidate_description_embeddings = [
            pytorch_utils.statement_embeddings[pytorch_utils.statement_item_id_to_row[candidate_id]] if candidate_id in pytorch_utils.statement_item_id_to_row
            else pytorch_utils.description_embeddings[pytorch_utils.description_item_id_to_row[candidate_id]]
            for candidate_id in candidate_ids
        ]

        
        # pad candidate_ids with 0s
        candidate_ids += [0] * (15 - len(candidate_ids))
        candidate_description_embeddings += torch.zeros(15 - len(candidate_description_embeddings), 384)

        candidate_description_embeddings = torch.stack(candidate_description_embeddings)
        

        batch_context_embeddings.append(context_embeddings[index])
        batch_entity_embeddings.append(entity_embeddings[index])
        candidate_ids_batch.append(candidate_ids)
        candidate_description_embeddings_batch.append(candidate_description_embeddings)

    # batch_context_embeddings is a list of tensors thus we have to use stack
    batch_context_embeddings = torch.stack(batch_context_embeddings)
    batch_entity_embeddings = torch.stack(batch_entity_embeddings)
    # candidate_ids_batch is a list of lists, convert it to a tensor
    candidate_ids_batch = torch.tensor(candidate_ids_batch, dtype=torch.long, device=device)
    candidate_description_embeddings_batch = torch.stack(candidate_description_embeddings_batch)

    # move everything to device
    batch_context_embeddings = batch_context_embeddings.to(device)
    batch_entity_embeddings = batch_entity_embeddings.to(device)
    candidate_ids_batch = candidate_ids_batch.to(device)
    candidate_description_embeddings_batch = candidate_description_embeddings_batch.to(device)

    return batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_train(x, device=device, entity_embeddings=dataset.entity_embeddings,context_embeddings=dataset.context_embeddings,syntax_candidates_list=dataset.syntax_candidates))
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_test(x, device=device, entity_embeddings=test_dataset.entity_embeddings,context_embeddings=test_dataset.context_embeddings,syntax_candidates_list=test_dataset.syntax_candidates))

In [ ]:
class EntityClassifier(torch.nn.Module):
    def __init__(self, device='cuda'):
        # This sequential layer will calculate a probability for each candidate entity, 
        # Since we will have CrossEntropyLoss as our loss function, we don't need to apply softmax
        # The input to this layer will be the concatenation of the sentence embedding, entity embedding, and candidate description embedding
        # The output will be single number, since it will be the probability of the candidate entity being the correct entity
        # We pass each candidate once and get the probability of it being the correct entity
        # Select the one with the highest probability
        super().__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(384*2 * 4, 384*2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(384*2, 1)
        )
        self.device = device
        self.to(device)
        
    def forward(self, x):
        batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch = x
        # The shapes are:
        # batch_context_embeddings: (batch_size, 384)
        # batch_entity_embeddings: (batch_size, 384)
        # candidate_ids_batch: (batch_size, 8)
        # candidate_description_embeddings_batch: (batch_size, 8, 384)

        context_entity_embed = torch.cat((batch_context_embeddings, batch_entity_embeddings), dim=1).to(self.device)
        mean_embed = (batch_context_embeddings + batch_entity_embeddings) / 2


        candidate_preds = torch.zeros((candidate_ids_batch.shape[0], candidate_ids_batch.shape[1]), device=self.device)
        for i in range(candidate_ids_batch.shape[1]):
            # Get the current candidate of all the batches
            candidate_description_embed = candidate_description_embeddings_batch[:,i,:]
            # The shape of candidate_description_embed is (batch_size, 384)
            # Concatenate the sentence_document_entity_embed and candidate_description_embed
            candidate_embed = torch.cat((context_entity_embed, candidate_description_embed), dim=1)
            difference_embed = torch.abs(mean_embed - candidate_description_embed)
            candidate_embed = torch.cat((candidate_embed, difference_embed), dim=1)
            # The shape of candidate_embed is (batch_size, 384 * 3)
            # Pass it through the sequential layer
            candidate_pred = self.sequential(candidate_embed)
            # The shape of candidate_pred is (batch_size, 1)
            # we squeeze to get into shape (batch_size,)
            candidate_preds[:,i] = candidate_pred.squeeze()
        return candidate_preds

In [ ]:
model = EntityClassifier(device=device)

In [ ]:
wiki_items = pd.read_csv(DATA_DIR + 'wiki_lite/wiki_items.csv')
# index wiki_items by id
wiki_items = wiki_items.set_index('item_id')
# Create item_id to wikipedia_title map
item_id_to_title = wiki_items['wikipedia_title'].to_dict()

enwiki_redirects = pd.read_csv(DATA_DIR + 'wiki_lite/enwiki_redirects.tsv', sep='\t', header=None, names=['source', 'target'])
# index enwiki_redirects by source
enwiki_redirects = enwiki_redirects.set_index('source')
# create source to target map
source_to_target = enwiki_redirects['target'].to_dict()


def get_predictions():
    # let's do predictions on the test set
    model.eval()
    with torch.no_grad():
        predictions = []
        prediction_indexes = []
        for i, data in enumerate(test_dataloader):
            batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch = data
            outputs = model((batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch))
            # which index has the highest value?
            _, predicted = torch.max(outputs, 1)
            prediction_indexes.extend(predicted.tolist())
            # predicted is shape (batch_size)
            # we want to grab the best candidate for batch item i
            # i.e the candidate_ids is of shape (batch_size, 5)
            # we want to grab the best candidate for batch item i
            # i.e get it to shape (batch_size)
            best_candidates = candidate_ids_batch[torch.arange(candidate_ids_batch.size(0)), predicted]

            # Append the best candidates to the predictions list
            predictions.extend(best_candidates.tolist())

    wiki_urls = []
    not_found = 0
    found = 0
    # Now we will map these into wikipedia_urls
    for i in range(len(predictions)):
        if predictions[i] == 0:
            # if the prediction is 0, we will append a blank url
            wiki_urls.append('NOT_FOUND')
            not_found += 1
            continue
        wikipedia_title = item_id_to_title[predictions[i]]
        # does this wikipedia title exist in the redirects?
        if wikipedia_title in source_to_target:
            # if it does, we will replace it with the redirect
            new_title = source_to_target[wikipedia_title]
            wikipedia_title = new_title
        # Now replace the spaces with underscores
        wikipedia_title = wikipedia_title.replace(' ', '_')
        # And add the wikipedia url
        wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')
        found += 1

    # print(f'Found {found} wikipedia urls')
    # print(f'Not found {not_found} wikipedia urls')
    # print(f'Percentage of wikipedia urls found: {found / (found + not_found)}')
    
    return wiki_urls
        

In [ ]:
validation = pd.read_csv(DATA_DIR + 'validation_prepped_with_item_id.csv')
ground_truth = validation['2'].to_list()


# loss and optimizer
from torch import optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

# training loop
from tqdm import tqdm


epochs = 20
pbar = tqdm(range(epochs))
highest_f1 = 0.0
best_epoch = 0
pbar.set_description(f'Epoch 0 loss: 0.0 Avg shape: 0.0 | Valid F1: 0.0')
for epoch in pbar:
    running_loss = 0.0
    valid_f1 = 0.0
    shapes = []
    model.train()
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch, labels_batch = data
        shapes.append(batch_context_embeddings.shape[0])
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model((batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch))
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()

    wiki_urls = get_predictions()
    # f1 from scikit-learn
    from sklearn.metrics import f1_score
    valid_f1 = f1_score(ground_truth, wiki_urls, average='micro')
    pbar.set_description(f'Epoch {epoch} loss: {running_loss / len(dataloader)} Avg shape: {sum(shapes) / len(shapes)} | Valid F1: {valid_f1}')
    pbar.update()

    if valid_f1 > highest_f1: # type: ignore
        highest_f1 = valid_f1
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_model3.pt')

print(f'Highest F1: {highest_f1} at epoch {best_epoch}')

Epoch 0 loss: 0.0 Avg shape: 0.0 | Valid F1: 0.0:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 19 loss: 0.19278715625861603 Avg shape: 14.799650043744531 | Valid F1: 0.7848570805149466: 100%|██████████| 20/20 [06:09<00:00, 18.45s/it]

Highest F1: 0.7902029238490073 at epoch 17


In [ ]:
# loss and optimizer
from torch import optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

# training loop
from tqdm import tqdm


epochs = 50
pbar = tqdm(range(epochs))
highest_f1 = 0.0
best_epoch = 0
for epoch in pbar:
    running_loss = 0.0
    valid_f1 = 0.0
    shapes = []
    model.train()
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch, labels_batch = data
        shapes.append(batch_context_embeddings.shape[0])
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model((batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch))
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()

    wiki_urls = get_predictions()
    # f1 from scikit-learn
    from sklearn.metrics import f1_score
    valid_f1 = f1_score(ground_truth, wiki_urls, average='micro')
    pbar.set_description(f'Epoch {epoch} loss: {running_loss / len(dataloader)} Avg shape: {sum(shapes) / len(shapes)} | Valid F1: {valid_f1}')
    pbar.update()

    if valid_f1 > highest_f1: # type: ignore
        highest_f1 = valid_f1
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_model.pt')

print(f'Highest F1: {highest_f1} at epoch {best_epoch}')

Epoch 4 loss: 0.17278752017934088 Avg shape: 14.799650043744531 | Valid F1: 0.7879118481344097:  10%|█         | 5/50 [01:43<15:34, 20.76s/it]


KeyboardInterrupt: 

In [ ]:
# load the best model
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [ ]:
wiki_urls = get_predictions()

In [ ]:
test = pd.read_csv(DATA_DIR + 'test.csv')
# train = pd.read_csv(DATA_DIR + 'train.csv')

In [ ]:
not_nan = test['wiki_url'].notna()
not_nme = test['wiki_url'] != '--NME--'
# train_not_nan = train['wiki_url'].notna()
# train_not_nme = train['wiki_url'] != '--NME--'
test.loc[(not_nan & not_nme) & (test.id == 65002)]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,?


In [ ]:
test.loc[not_nan & not_nme, 'wiki_url'] = wiki_urls
# train.loc[train_not_nan & train_not_nme, 'wiki_url'] = train_wiki_urls

In [ ]:
# replace NaN or --NME-- with NOT_FOUND
test['wiki_url'] = test['wiki_url'].fillna('NOT_FOUND')
test['wiki_url'] = test['wiki_url'].replace('--NME--', 'NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].fillna('NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].replace('--NME--', 'NOT_FOUND')

In [ ]:
test[test.id == 65002]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,http://en.wikipedia.org/wiki/Dejan_Koturović


In [ ]:
' '.join(test.token.fillna('', inplace=False).to_list()[:250])

"-DOCSTART- (947testa CRICKET) CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .  LONDON 1996-08-30  West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .  Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .  After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .  Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .  Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshi

In [ ]:
# # One problem, Simmons does not retrieve the cricket player, but querying for Phil Simmons does.
# # TODO: Fix this, one idea is to group by per doc id, check if this token has a better previous full mention
# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['simmons'])])

# print('____')

# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['phil simmons'])])

In [ ]:
# now create a .csv file from id, wiki_url
test[['id', 'wiki_url']].to_csv('submission_bge.csv', index=False)
# train[['id', 'wiki_url']].to_csv('train_with_doc.csv', index=False)